In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import scipy.io
import cv2
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import torch.utils.data
%matplotlib inline



/home/bencusb/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

df=pd.read_csv('/run/media/bencusb/Windows_7_Ultimate_32_Bit/mpii_dataset.csv')



In [5]:
from sklearn.model_selection import train_test_split
df.drop(['Category','Activity'],1)
df = df.sample(n = len(df)).reset_index(drop=True)

/tmp/ipykernel_57220/934581437.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Category','Activity'],1)


In [6]:
df.head()
from torch.utils.data import Dataset


In [7]:
transferdata_df = df.iloc[- 1400:, 0:]

test_df = pd.DataFrame()
test_df= test_df.append(transferdata_df)
train_df =df.drop(transferdata_df.index)
test_df = test_df.reset_index(drop=True)
test_df.head()

/tmp/ipykernel_57220/1936806666.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df= test_df.append(transferdata_df)


,Unnamed: 0,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,0,065131990.jpg,422,370,421,268,449,157,489,165,...,93,415,117,419,217,414,286,2.458800,"diving, springboard or platform",water activities
1,0,095640078.jpg,707,593,712,516,643,505,655,506,...,420,685,434,663,474,658,525,1.565161,skateboarding,sports
2,0,094069019.jpg,231,312,245,340,227,267,267,264,...,173,286,174,300,220,317,243,1.590894,"dusting or polishing furniture, general",home activities
3,0,025178845.jpg,-1,-1,-1,-1,557,355,511,369,...,203,424,222,402,302,379,284,2.257149,scraping and painting sailboat or powerboat,home repair
4,0,068931041.jpg,1041,302,991,500,903,764,975,841,...,670,748,801,860,657,849,548,5.088906,rock climbing,sports


In [8]:
img_loc=("/run/media/bencusb/Windows_7_Ultimate_32_Bit/Images/images/")
#imgArray = np.array([])
#files = glob.glob(img_loc+'*.jpg')
imgcomb=[]


images=[]       
trainimglabels=train_df.loc[:,'r ankle_X':'Scale']
trainfilenames=train_df['NAME']

testimglabels=test_df.loc[:,'r ankle_X':'Scale']
testfilenames=test_df['NAME']

class Imagedataset(Dataset):
    def __init__(self,fnames,labels,transform=None):
        self.fnames=fnames
        self.labels=labels
        self.transform=transform
    
    def __len__(self):
        return len(self.fnames)
    
    def NormaliseData(self,img,idx):
        #print(self.labels.head())
        height, width = img.shape
        #print(self.labels.head())
        Xcoord=[]
        Ycoord=[]
        x=0
        for i in range(0,31,2):
            self.labels.iloc[idx,i]=(self.labels.iloc[idx,i]*(256/width))
            Xcoord.append(self.labels.iloc[idx,i])
            self.labels.iloc[idx,i+1]=(self.labels.iloc[idx,i+1]*(256/height))
            Ycoord.append(self.labels.iloc[idx,i+1])
            x=x+1
        X_min=np.min(Xcoord)
        Y_min=np.min(Ycoord)
        X_max=np.max(Xcoord)
        Y_max=np.max(Ycoord)
        x=0
        for i in range(0,31,2):
            self.labels.iloc[idx, i] = (self.labels.iloc[idx, i] - X_min) / (X_max - X_min)
            self.labels.iloc[idx, i+1] = (self.labels.iloc[idx, i+1] - Y_min) / (Y_max - Y_min)
            x=x+1
        #print(self.labels.head())
        normImg=cv2.resize(img,(256,256))
        return normImg


    def __getitem__ (self, idx):
        Rimg=cv2.imread(img_loc+self.fnames[idx])
        Gimg=cv2.cvtColor(Rimg,cv2.COLOR_BGR2GRAY)
        Gimg=self.NormaliseData(Gimg,idx)
        #for i in range(0,31,2):
         #   y=self.labels.iloc[idx,i+1]
          #  x=self.labels.iloc[idx,i]
           # cv2.circle(Gimg,(int(x),int(y)),5,(0,255,0),-1)
        #cv2.imshow("gay", Gimg)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        Gimg = Gimg.astype(np.float32) / 255.
        if self.transform:
            Gimg = self.transform(Gimg)
       
        helper=self.labels.iloc[idx, :-1]
       
        target=helper.values
        target=target.reshape(-1,2)
     
        return (Gimg,target)

    
train_dataset= Imagedataset(trainfilenames,trainimglabels)
test_dataset= Imagedataset(testfilenames,testimglabels)

train_loader=DataLoader(train_dataset,batch_size=15, shuffle=True, num_workers=4)
test_loader=DataLoader(test_dataset,batch_size=15, shuffle=True)




In [9]:
data, target= train_dataset.__getitem__(1)
asdf=pd.DataFrame(data)
print(data.shape)


(256, 256)


In [10]:
train_dataset.fnames.iloc[4]
import itertools

In [11]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1)
        self.conv4 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1)
        self.conv5 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1)
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(128*6*6,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,16*2)

    def forward(self, X):
        X = self.conv1(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv2(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv3(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv3(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv4(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = torch.flatten(X,1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = F.relu(X)
        X = self.fc4(X)
        X=X.view(-1,16,2)
        output = F.log_softmax(X, dim=1)
        return output
    
torch.manual_seed(101)
model = ConvolutionalNetwork().to(dev)
print(dev)

cuda:0


In [17]:

criterion = nn.MultiLabelSoftMarginLoss() #try diceloss, MultiLabelSoftMarginLoss
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [13]:
def train(model, dev, train_loader, optimizer, epoch):
  model.train()
  
  batch_size = 15
  for batch_idx in range(0,len(train_loader),batch_size):
    #print("dik")
    #start_idx = batch_idx
    #end_idx = min(len(train_loader), batch_idx + 20)
    batch=list(itertools.islice(train_loader,batch_size))
    for data, target in batch:
      #print("coc")
      data=data.unsqueeze(1)
      #print(data.shape ," MIAGECI ",target.shape)
      data, target = data.to(dev), target.to(dev)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
      epoch, batch_idx * len(data), len(train_loader.dataset),
      100. * batch_idx / len(train_loader), loss.item()))   
    
    
        

In [14]:

train_dataset.__getitem__(0)


(array([[0.9529412 , 0.9529412 , 0.9529412 , ..., 0.9529412 , 0.9529412 ,
         0.9529412 ],
        [0.9529412 , 0.9529412 , 0.9529412 , ..., 0.9529412 , 0.9529412 ,
         0.9529412 ],
        [0.9529412 , 0.9529412 , 0.9529412 , ..., 0.9529412 , 0.9529412 ,
         0.9529412 ],
        ...,
        [0.6862745 , 0.6862745 , 0.6862745 , ..., 0.6431373 , 0.6392157 ,
         0.64705884],
        [0.6666667 , 0.67058825, 0.67058825, ..., 0.6509804 , 0.6509804 ,
         0.64705884],
        [0.654902  , 0.654902  , 0.64705884, ..., 0.64705884, 0.61960787,
         0.56078434]], dtype=float32),
 array([[0.77363103, 0.93542079],
        [0.47180575, 0.73148646],
        [0.68739524, 0.43238277],
        [0.85986683, 0.4238855 ],
        [0.78081735, 0.74338263],
        [1.        , 1.        ],
        [0.77363103, 0.42898386],
        [0.48258522, 0.12648126],
        [0.4010446 , 0.10511014],
        [0.        , 0.        ],
        [0.55085523, 0.50545924],
        [0.50773733,

In [15]:
torch.cuda.empty_cache()

In [19]:
for epoch in range(1, 6):
    train(model, dev, train_loader, optimizer, epoch)

Train Epoch: 1 [0/15972 (0%)]	Loss: 1.490
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.588
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.414
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.517
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.576
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.397
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.418
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.438
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.424
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.433
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.594
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.511
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.473
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.521
Train Epoch: 1 [0/15972 (0%)]	Loss: 1.368
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.478
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.506
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.434
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.427
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.392
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.467
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.474
Train Epoch: 1 [225/15972 (1%)]	Loss: 1.516
Train Epoch: 1 [22

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f022a021240>
Traceback (most recent call last):
  File "/home/bencusb/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/bencusb/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1430, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.495
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.511
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.330
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.430
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.395
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.539
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.430
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.440
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.416
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.467
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.443
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.482
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.508
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.401
Train Epoch: 4 [13275/15972 (83%)]	Loss: 1.462
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.479
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.477
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.455
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.443
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.583
Train Epoch: 4 [13500/15972 (85%)]	Loss: 1.445
Train Epoch: 